In [1]:
import pandas as pd
dfTotal = pd.read_csv("../../refined/perfectDailyTotals.csv")
dfAcorn = pd.read_csv("../../refined/perfectCustomers.csv")

In [2]:
dfTotal.head()

,id,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,...,2013-12-22,2013-12-23,2013-12-24,2013-12-25,2013-12-26,2013-12-27,2013-12-28,2013-12-29,2013-12-30,2013-12-31
0,2,10.800,13.300,10.074,9.857,11.171,10.293,9.439000,11.640000,15.259,...,12.025,6.721,4.692,4.573,4.590,9.031,13.535,14.876,13.924,13.415000
1,6,0.500,0.496,0.494,0.495,0.499,0.498,0.489000,1.854000,3.184,...,3.865,3.815,4.306,4.932,3.864,3.398,2.887,3.047,3.862,3.261000
2,10,32.330,32.559,30.772,33.313,30.310,35.150,37.274002,37.500000,32.796,...,35.666,34.399,47.192,64.189,41.526,46.156,45.916,38.192,41.568,37.697998
3,18,11.789,14.018,14.675,17.501,15.830,22.351,17.674000,16.769001,10.459,...,9.037,9.085,8.911,8.908,13.687,12.605,11.750,14.956,14.143,9.387000
4,19,8.924,8.421,7.818,7.549,6.720,7.826,7.790000,9.333000,8.195,...,8.310,6.925,4.219,4.069,4.000,3.675,8.111,8.894,7.193,8.410000


In [3]:
X = dfTotal.drop(columns = ['id'])
y = (dfAcorn['acorn-grouped'] == 'Affluent').astype(int).values
z = dfAcorn['acorn-grouped']

target_names = ["Non-Affluent", "Affluent"]

In [4]:
# Perform a train-test split.

from sklearn.model_selection import train_test_split

# Default split ratio is test = 0.25, train = 0.75.
# The stratify=z ensures that we have pro-rata sampling from each acorn-group.

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=31, stratify=z)

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC 
from sklearn.svm import SVC 
from sklearn.metrics import classification_report
from collections import Counter

In [6]:
dfAff = dfAcorn.loc[dfAcorn['acorn-grouped']=='Affluent']
dfNonAff = dfAcorn.loc[dfAcorn['acorn-grouped']!='Affluent']
dfNonAff = dfNonAff.sample(n = len(dfAff))
dfEqAff = dfAff.append(dfNonAff)
dfEqAff = dfEqAff.sort_values(by=['id'])
dfEqual = pd.merge(dfEqAff['id'], dfTotal, how='left', on = 'id') 

In [7]:
X2 = dfEqual.drop(columns = ['id'])
y2 = (dfEqAff['acorn-grouped'] == 'Affluent').astype(int).values
z2 = dfEqAff['acorn-grouped']

target_names = ["Non-Affluent", "Affluent"]

# Perform a train-test split.

from sklearn.model_selection import train_test_split

# Default split ratio is test = 0.25, train = 0.75.
# The stratify=z ensures that we have pro-rata sampling from each acorn-group.

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=31, stratify=z2)

In [8]:
# Train using equal number of affluent and non-affluent.
# Test using proportions consistent with the original supports.

model = LinearSVC(verbose=3,  max_iter = 1000000, C=2)
model.fit(X_train2, y_train2)

predictions = model.predict(X_test)
print(Counter(y_test-predictions))
rat = Counter(y_train-predictions)[0] / sum(Counter(y_train-predictions).values())
print(round(rat,2))
print(classification_report(y_test, predictions, target_names=target_names))

[LibLinear]Counter({0: 171, -1: 30, 1: 20})
0.7737556561085973
              precision    recall  f1-score   support

Non-Affluent       0.84      0.77      0.80       133
    Affluent       0.69      0.77      0.73        88

   micro avg       0.77      0.77      0.77       221
   macro avg       0.77      0.77      0.77       221
weighted avg       0.78      0.77      0.78       221



<h2>Does the model work well with the 'perfect' data set ?</h2>

In [17]:
predictions = model.predict(X_train)
print(Counter(y_train-predictions))
rat = Counter(y_train-predictions)[0] / sum(Counter(y_train-predictions).values())
print(round(rat,2))
print(classification_report(y_train, predictions, target_names=target_names))

Counter({0: 542, -1: 95, 1: 23})
0.82
              precision    recall  f1-score   support

Non-Affluent       0.93      0.76      0.84       399
    Affluent       0.71      0.91      0.80       261

   micro avg       0.82      0.82      0.82       660
   macro avg       0.82      0.84      0.82       660
weighted avg       0.84      0.82      0.82       660



<h2>Does the model work well with the 'entire' data set ?</h2>

In [18]:
import pandas as pd
dfTotal = pd.read_csv("../../refined/DailyTotals.csv")
dfAcorn = pd.read_csv("../../refined/entireCustomers.csv")

X = dfTotal.drop(columns = ['id'])
y = (dfAcorn['acorn-grouped'] == 'Affluent').astype(int).values

predictions = model.predict(X)
print(Counter(y-predictions))
rat = Counter(y-predictions)[0] / sum(Counter(y-predictions).values())
print(round(rat,2))
print(classification_report(y, predictions, target_names=target_names))


Counter({0: 2549, -1: 1139, 1: 723})
0.58
              precision    recall  f1-score   support

Non-Affluent       0.69      0.58      0.63      2724
    Affluent       0.46      0.57      0.51      1687

   micro avg       0.58      0.58      0.58      4411
   macro avg       0.57      0.58      0.57      4411
weighted avg       0.60      0.58      0.58      4411

